In [ ]:
import numpy as np
import cv2
import os

 
images_folder = './N2-embryo-10um-SRx5-crop-images-correct-angle-folders'
seg_folder = './N2-embryo-10um-SRx5-crop-seg-correct-angle-folders'

sub_folders = os.listdir(images_folder)

print(sub_folders)

pointcloud_folder = "N2-embryo-10um-SRx5-3D-pointcloud"


try:
    os.makedirs(pointcloud_folder)
    print(f"Folder '{pointcloud_folder}' created successfully.")
except FileExistsError:
    print(f"Folder '{pointcloud_folder}' already exists.")   



In [ ]:
import math
import open3d
from copy import deepcopy


for sub_folder in sub_folders:

    images_list_dir = images_folder+'/'+sub_folder
    seg_images_list_dir = seg_folder+'/'+sub_folder

    images_list = os.listdir(images_list_dir)
    seg_images_list = os.listdir(seg_images_list_dir)


    
    if '13' in sub_folder or '19' in sub_folder or '20' in sub_folder or '24' in sub_folder:
        continue

    
    
    print(sub_folder)

    
    best_match_index = 0
    best_match_score = 0
    for image_name in images_list:

        if image_name.endswith('.png'):
            image_index = int(image_name.replace('.png',''))
            
            if image_index==0:

                img_tempate = cv2.imread(images_list_dir + '/' + image_name, cv2.IMREAD_GRAYSCALE)

            elif image_index>500:
                img = cv2.imread(images_list_dir + '/' + image_name, cv2.IMREAD_GRAYSCALE)

                res = cv2.matchTemplate(img, img_tempate, cv2.TM_CCOEFF_NORMED)

                score = res[0][0]
                if score > best_match_score:
                    best_match_index = image_index
                    best_match_score = score
                    

    print('best_match_index = ',best_match_index, ', best_match_score = ' ,best_match_score)


    
    cloud_points = None
    degree_per_frame = 360.0/(best_match_index+1)

    for image_name in images_list:

        if image_name.endswith('.png'):
            image_index = int(image_name.replace('.png',''))
            
            
            if image_index >= 0 and image_index <= best_match_index:

                mask = cv2.imread(seg_images_list_dir + '/' + image_name, cv2.IMREAD_GRAYSCALE)

                h,w = mask.shape


                
                contours = cv2.findContours(
                    mask.astype("uint8"), cv2.RETR_CCOMP,
                    cv2.CHAIN_APPROX_NONE)[-2]

                x,y,w,h = cv2.boundingRect(contours[0])

                rotate_axis = w/2-1

                rotate_axis = rotate_axis+x


                theta = 1.0*math.radians((image_index)*degree_per_frame)
                contour_points = np.zeros((contours[0].shape[0],3))
                contour_points[:,0:2] = contours[0].reshape(-1,2)

                
                new_contours = contour_points[contour_points[:,0]<241]
                

                center_y = y+(w/2)
                delta_y = 360-center_y
                

                pcd = open3d.geometry.PointCloud()
                pcd.points = open3d.utility.Vector3dVector(new_contours)

                R = pcd.get_rotation_matrix_from_xyz((0, theta, 0))
                pcd.rotate(R,center=(rotate_axis, 0, 0)) 

                if cloud_points == None:
                    cloud_points = pcd
                else:

                    cloud_points.paint_uniform_color([1,1,1])

                    pcd.paint_uniform_color([1,0,0])
                    cloud_points += pcd

    
    


    
    origin_pcd = deepcopy(cloud_points)
    pcd2 = deepcopy(origin_pcd)

    for image_name in images_list:

        if image_name.endswith('.png'):
            image_index = int(image_name.replace('.png',''))
            

            if image_index % int(best_match_index/6) != 0 and image_index !=best_match_index:
                continue


            
            if image_index >= 0 and image_index <= best_match_index:
                
                img  = cv2.imread(images_list_dir + '/' + image_name) 
                mask = cv2.imread(seg_images_list_dir + '/' + image_name, cv2.IMREAD_GRAYSCALE) 



                
                contours = cv2.findContours(
                    mask.astype("uint8"), cv2.RETR_CCOMP,
                    cv2.CHAIN_APPROX_NONE)[-2]

                img_origin = deepcopy(img)
                cv2.drawContours(img, contours, -1, (0, 0, 0), 25)

                
                
                

                

                

                offset_x,offset_y,w,h = cv2.boundingRect(contours[0])
                
                
                rotate_axis = w/2-1
                rotate_axis = rotate_axis+offset_x

                theta = -1.0*math.radians((image_index)*0.566037735849) 
                R = pcd2.get_rotation_matrix_from_xyz((0, theta, 0))
                pcd2.rotate(R, center=(rotate_axis, 0, 0))
                origin_pcd.rotate(R, center=(rotate_axis, 0, 0))

                
                projection_list = []
                projection = np.zeros_like(img)

                

                for i in range(len(pcd2.points)):
                    x, y, z  = pcd2.points[i]
                    color = pcd2.colors[i]
                    if z > 0:
                        projection_list.append([x,y])
                        x = int(x)
                        y = int(y)
                        if x >= 480:
                            x = 479
                        if x <0:
                            x=0

                        if y >= 720:
                            y = 719
                        if y <0:
                            y=0

                        if np.mean(img[y,x]) > 0:

                            
                            

                            
                            
                            pcd2.colors[i] = img[y,x]/255
    
    
    open3d.io.write_point_cloud(pointcloud_folder+'/'+sub_folder+'_origin.ply', origin_pcd , write_ascii=True)  
    open3d.io.write_point_cloud(pointcloud_folder+'/'+sub_folder+'_tietu.ply', pcd2 , write_ascii=True)  